In [79]:
from pathlib import Path
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image
import hsn_v1

In [80]:
# path = 'img/02_glas_full'
path = 'img/02_glas_patch'

In [81]:
glas_paths = paths = [str(path) for path in Path(path).rglob('*.png')]

In [82]:
glas_paths

['img/02_glas_patch/train_20_r1_c0.png',
 'img/02_glas_patch/testA_8_r0_c1.png',
 'img/02_glas_patch/train_25_r0_c0.png',
 'img/02_glas_patch/testA_57_r1_c1.png',
 'img/02_glas_patch/train_19_r0_c0.png',
 'img/02_glas_patch/train_63_r1_c1.png',
 'img/02_glas_patch/testA_35_r1_c1.png',
 'img/02_glas_patch/testA_40_r0_c1.png',
 'img/02_glas_patch/testA_23_r0_c1.png',
 'img/02_glas_patch/train_64_r1_c1.png',
 'img/02_glas_patch/train_51_r0_c1.png',
 'img/02_glas_patch/testA_27_r0_c0.png',
 'img/02_glas_patch/testB_7_r0_c0.png',
 'img/02_glas_patch/testA_10_r0_c1.png',
 'img/02_glas_patch/train_23_r0_c0.png',
 'img/02_glas_patch/train_40_r0_c1.png',
 'img/02_glas_patch/train_82_r1_c1.png',
 'img/02_glas_patch/train_66_r1_c0.png',
 'img/02_glas_patch/testA_16_r1_c1.png',
 'img/02_glas_patch/testA_46_r0_c1.png',
 'img/02_glas_patch/train_74_r1_c0.png',
 'img/02_glas_patch/testA_5_r1_c1.png',
 'img/02_glas_patch/testA_52_r0_c1.png',
 'img/02_glas_patch/testA_53_r0_c0.png',
 'img/02_glas_patch

In [83]:
glas_test = [file for file in glas_paths if 'test' in file]
glas_train = [file for file in glas_paths if 'train' in file]

In [84]:
X_train = []
for file in glas_train:
    img = Image.open(file)
    X_train.append(np.array(img))
X_train = np.array(X_train)

In [85]:
X_test = []
for file in glas_test:
    img = Image.open(file)
    X_test.append(np.array(img))
X_test = np.array(X_test)

In [86]:
X_train.shape, X_test.shape

((340, 224, 224, 3), (320, 224, 224, 3))

### Prepare labels

In [87]:
GO_INDEX = 48

In [88]:
y_train = np.zeros((X_train.shape[0], 51))
y_train[:,GO_INDEX] = 1

In [89]:
y_test = np.zeros((X_test.shape[0], 51))
y_test[:,GO_INDEX] = 1

In [90]:
# Validation set split 
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
X_train.shape, X_val.shape

((272, 224, 224, 3), (68, 224, 224, 3))

### Load model

In [91]:
MODEL_NAME = 'histonet_X1.7_clrdecay_5'
INPUT_NAME = '02_glas_full'
INPUT_MODE = 'patch'                    # {'patch', 'wsi'}
INPUT_SIZE = [224, 224]                 # [<int>, <int>] > 0
HTT_MODE = 'glas'                       # {'both', 'morph', 'func', 'glas'}
BATCH_SIZE = 1                          # int > 0
GT_MODE = 'on'                          # {'on', 'off'}
RUN_LEVEL = 3                           # {1: HTT confidence scores, 2: Grad-CAMs, 3: Segmentation masks}
SAVE_TYPES = [1, 1, 1, 1]               # {HTT confidence scores, Grad-CAMs, Segmentation masks, Summary images}
VERBOSITY = 'QUIET'                    # {'NORMAL', 'QUIET'}
# Settings for image set
IN_PX_RESOL = 0.620
OUT_PX_RESOL = 0.25 * 1088 / 224    # 1.21428571429
DOWNSAMPLE_FACTOR = OUT_PX_RESOL / IN_PX_RESOL

In [92]:
hsn = hsn_v1.HistoSegNetV1(params={'input_name': INPUT_NAME, 'input_size': INPUT_SIZE, 'input_mode': INPUT_MODE,
                                       'down_fac': DOWNSAMPLE_FACTOR, 'batch_size': BATCH_SIZE, 'htt_mode': HTT_MODE,
                                       'gt_mode': GT_MODE, 'run_level': RUN_LEVEL, 'save_types': SAVE_TYPES,
                                       'verbosity': VERBOSITY})

In [93]:
hsn.load_histonet(params={'model_name': MODEL_NAME})

In [94]:
histonet = hsn.hn
histonet.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 64)      1792      
_________________________________________________________________
activation_1 (Activation)    (None, 224, 224, 64)      0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 224, 224, 64)      256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 224, 224, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
activation_2 (Activation)    (None, 224, 224, 64)      0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 224, 224, 64)      256       
__________

### Train model

In [95]:
model = histonet.model

In [96]:
history = model.fit(X_train, y_train, epochs=16,
                    validation_data=(X_val, y_val))

Instructions for updating:
Use tf.cast instead.
Train on 272 samples, validate on 68 samples
Epoch 1/16


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_1_1/convolution}}]]
	 [[{{node loss_1/add_7}}]]